# Text Preprocessing

## 1 Import Library & Load Data

In [1]:
import pandas as pd 
import numpy as np
import string 
import re #regex library

# import word_tokenize & FreqDist from NLTK
from nltk.tokenize import word_tokenize 
from nltk.probability import FreqDist

In [2]:

YOUTUBE_COMMENTS = pd.read_csv("../dataset/raw/vaksin_covid_13-12-2020.csv")
YOUTUBE_COMMENTS.tail(10)

,Video ID,Title,Comment,Label
1856,HjizKEjym0g,"UPDATE KOMITE KPCPEN - Vaksin Covid-19, Protok...",700 triliun ??? Kagak pernah trima.,NaN
1857,HjizKEjym0g,"UPDATE KOMITE KPCPEN - Vaksin Covid-19, Protok...",New world order dimulai . . .,NaN
1858,HjizKEjym0g,"UPDATE KOMITE KPCPEN - Vaksin Covid-19, Protok...",Hindari kerumunan . Hindari demo 😂😂😂,NaN
1859,HjizKEjym0g,"UPDATE KOMITE KPCPEN - Vaksin Covid-19, Protok...",Lebbay,NaN
1860,HjizKEjym0g,"UPDATE KOMITE KPCPEN - Vaksin Covid-19, Protok...",Negara aneh...soal kesehatan ditangani tentara...,NaN
1861,HjizKEjym0g,"UPDATE KOMITE KPCPEN - Vaksin Covid-19, Protok...",Alhamdulillah saya bersyukur sekali karena You...,NaN
1862,HjizKEjym0g,"UPDATE KOMITE KPCPEN - Vaksin Covid-19, Protok...","Gak ada itu corona..,terlalu berlebihan kalian",NaN
1863,HjizKEjym0g,"UPDATE KOMITE KPCPEN - Vaksin Covid-19, Protok...",56:30 band padi reborn,NaN
1864,HjizKEjym0g,"UPDATE KOMITE KPCPEN - Vaksin Covid-19, Protok...",Bagus semoga corona cepat berlalu. Semoga duni...,NaN
1865,NaN,NaN,NaN,NaN


# Case Folding

## Lower Case

In [3]:
YOUTUBE_COMMENTS['comment_clean'] = YOUTUBE_COMMENTS['Comment'].str.lower()
# print(YOUTUBE_COMMENTS['comment_clean'].head())

## Remove Special Text 

In [4]:
def remove_comment_special(text):
    # remove tab, new line, ans back slice
    text = str(text).replace('\\t'," ").replace('\\n'," ").replace('\\u'," ").replace('\\',"")
    # remove non ASCII (emoticon, chinese word, .etc)
    text = text.encode('ascii', 'replace').decode('ascii')
    # remove mention, link, hashtag
    text = ' '.join(re.sub("([@#][A-Za-z0-9]+)|(\w+:\/\/\S+)"," ", text).split())
    # remove incomplete URL
    return text.replace("http://", " ").replace("https://", " ")

In [5]:
YOUTUBE_COMMENTS['comment_clean'] = YOUTUBE_COMMENTS['comment_clean'].apply(remove_comment_special)
# YOUTUBE_COMMENTS['comment_clean'].head(10)

## Remove Number 

In [6]:
def remove_number(text):
    return  re.sub(r"\d+", "", text)

In [7]:
YOUTUBE_COMMENTS['comment_clean'] = YOUTUBE_COMMENTS['comment_clean'].apply(remove_number)
# YOUTUBE_COMMENTS['comment_clean'].head(10)

## Remove Punctuation

In [8]:
def remove_punctuation(text):
    return text.translate(str.maketrans(string.punctuation, ' '*len(string.punctuation)))


In [9]:
YOUTUBE_COMMENTS['comment_clean'] = YOUTUBE_COMMENTS['comment_clean'].apply(remove_punctuation)
# YOUTUBE_COMMENTS['comment_clean'].head(50)

## Remove Whitespace

In [10]:
#remove whitespace leading & trailing
def remove_whitespace_LT(text):
    return text.strip()

In [11]:
YOUTUBE_COMMENTS['comment_clean'] = YOUTUBE_COMMENTS['comment_clean'].apply(remove_whitespace_LT)
# YOUTUBE_COMMENTS['comment_clean'].head(50)

## Remove Multiple Whitespace Into Single Whitespace

In [12]:
def remove_whitespace_multiple(text):
    return re.sub('\s+',' ',text)

In [13]:
YOUTUBE_COMMENTS['comment_clean'] = YOUTUBE_COMMENTS['comment_clean'].apply(remove_whitespace_multiple)
# YOUTUBE_COMMENTS['comment_clean'].head(50)

## Remove Single Character

In [14]:
def remove_singl_char(text):
    return re.sub(r"\b[a-zA-Z]\b", "", text)

In [15]:
YOUTUBE_COMMENTS['comment_clean'] = YOUTUBE_COMMENTS['comment_clean'].apply(remove_singl_char)

## Show Case Folding Result

In [16]:
YOUTUBE_COMMENTS['comment_clean'].head(10)

0                                   vaksin tak berguna
1    penelitian vaksin yg bgtu cpat vaksin dbd sars...
2    klo bs faksinnya di tambahin odading mang oleh...
3    yg divaksin pertama harusnya orang nomor di ri...
4                                   tolak vaksin covid
5    yg pertama disuntik vaksin januari adalah pres...
6                            mohon jgn beli vaksin pak
7    harap nya vaksin tak berjaya nanti auto kaya k...
8    efek samping otot kejang demam tinggi batuk keras
9    lol vaksin darimana ini korona dalam hitungan ...
Name: comment_clean, dtype: object

# Tokenization

## NLTK Word Tokenize

In [17]:
# NLTK word tokenize 
def word_tokenize_wrapper(text):
    return word_tokenize(text)

In [18]:
YOUTUBE_COMMENTS['comment_clean'] = YOUTUBE_COMMENTS['comment_clean'].apply(word_tokenize_wrapper)

In [19]:
# print(YOUTUBE_COMMENTS['comment_clean'].head())

## Frequency distribution

In [20]:
# NLTK calc frequency distribution
def freqDist_wrapper(text):
    return FreqDist(text)

YOUTUBE_COMMENTS['comment_freq_token'] = YOUTUBE_COMMENTS['comment_clean'].apply(freqDist_wrapper)

print('Frequency Tokens : \n') 
print(YOUTUBE_COMMENTS['comment_freq_token'].head().apply(lambda x : x.most_common()))

Frequency Tokens : 

0                [(vaksin, 1), (tak, 1), (berguna, 1)]
1    [(vaksin, 2), (penelitian, 1), (yg, 1), (bgtu,...
2    [(bs, 2), (biar, 2), (klo, 1), (faksinnya, 1),...
3    [(pertama, 2), (yg, 1), (divaksin, 1), (harusn...
4                [(tolak, 1), (vaksin, 1), (covid, 1)]
Name: comment_freq_token, dtype: object


# Normalization

In [21]:
normalizad_word = pd.read_excel("../normalisasi.xlsx")

normalizad_word_dict = {}

for index, row in normalizad_word.iterrows():
    if row[0] not in normalizad_word_dict:
        normalizad_word_dict[row[0]] = row[1] 

def normalized_term(document):
    return [normalizad_word_dict[term] if term in normalizad_word_dict else term for term in document]

YOUTUBE_COMMENTS['comment_normal'] = YOUTUBE_COMMENTS['comment_clean'].apply(normalized_term)

# YOUTUBE_COMMENTS['comment_clean'].head()

In [22]:
YOUTUBE_COMMENTS['comment_normal_joined'] = YOUTUBE_COMMENTS['comment_normal'].str.join(' ')

# Stopword Removal

In [23]:
from nltk.corpus import stopwords
# get stopword indonesia
list_stopwords = stopwords.words('indonesian')

list_stopwords.extend(["yg", "dg", "rt", "dgn", "ny", "d", 'klo', 
                       'kalo', 'amp', 'biar', 'bikin', 'bilang', 
                       'gak', 'ga', 'krn', 'nya', 'nih', 'sih', 
                       'si', 'tau', 'tdk', 'tuh', 'utk', 'ya', 
                       'jd', 'jgn', 'sdh', 'aja', 'n', 't', 
                       'nyg', 'hehe', 'pen', 'u', 'nan', 'loh', 'rt',
                       '&amp', 'yah'])

In [24]:
# read txt stopword using pandas
txt_stopword = pd.read_csv("../stopwords.txt", names= ["stopwords"], header = None)

In [25]:
list_stopwords.extend(txt_stopword["stopwords"][0].split(' '))

In [26]:
# convert list to dictionary
list_stopwords = set(list_stopwords)

In [27]:
#remove stopword pada list token
def stopwords_removal(words):
    return [word for word in words if word not in list_stopwords]

In [28]:
YOUTUBE_COMMENTS['comment_stopword'] = YOUTUBE_COMMENTS['comment_normal'].apply(stopwords_removal) 

In [29]:
 print(YOUTUBE_COMMENTS['comment_stopword'].head())

0                                    [vaksin, berguna]
1    [penelitian, vaksin, cepat, vaksin, dbd, sars,...
2    [faksinnya, tambahin, odading, mang, ironman, ...
3    [divaksin, orang, nomor, ri, presiden, suntik,...
4                               [tolak, vaksin, covid]
Name: comment_stopword, dtype: object


# Stemming

In [30]:
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

In [31]:
# create stemmer
factory = StemmerFactory()
stemmer = factory.create_stemmer()

In [32]:
# stemmed
def stemmed_wrapper(term):
    return stemmer.stem(term)

term_dict = {}

In [33]:
for document in YOUTUBE_COMMENTS['comment_stopword']:
    for term in document:
        if term not in term_dict:
            term_dict[term] = ' '
            
print(len(term_dict))
print("------------------------")

5245
------------------------


In [34]:
for term in term_dict:
    term_dict[term] = stemmed_wrapper(term)
    print(term,":" ,term_dict[term])
    
print(term_dict)
print("------------------------")

vaksin : vaksin
berguna : guna
penelitian : teliti
cepat : cepat
dbd : dbd
sars : sars
dll : dll
tmukan : tmukan
faksinnya : faksinnya
tambahin : tambahin
odading : odading
mang : mang
ironman : ironman
penyakit : sakit
nyerang : nyerang
divaksin : vaksin
orang : orang
nomor : nomor
ri : ri
presiden : presiden
suntik : suntik
rakyat : rakyat
percaya : percaya
aman : aman
coba : coba
doang : doang
tolak : tolak
covid : covid
disuntik : suntik
januari : januari
kantor : kantor
stap : stap
anggouta : anggouta
dpr : dpr
elit : elit
politik : politik
mentri : tri
polri : polri
tni : tni
asn : asn
masyarakat : masyarakat
diatas : atas
pa : pa
mohon : mohon
beli : beli
harap : harap
berjaya : jaya
auto : auto
kaya : kaya
wkwk : wkwk
efek : efek
samping : samping
otot : otot
kejang : kejang
demam : demam
batuk : batuk
keras : keras
lol : lol
darimana : darimana
korona : korona
hitungan : hitung
vaksinnya : vaksin
kabar : kabar
kelinci : kelinci
percobaan : coba
china : china
no : no
mah : mah


dokter : dokter
menangani : tangan
rs : rs
nyawa : nyawa
dikurangin : rangin
dikerokin : dikerokin
wapres : wapres
dprd : dprd
bos : bos
brhasil : brhasil
us : us
rusia : rusia
memilih : pilih
hubungan : hubung
diplomatik : diplomatik
soekarno : soekarno
lihat : lihat
berskala : skala
perintah : perintah
panutan : panutan
pakde : pakde
dipikirin : dipikirin
mulu : mulu
tagihan : tagih
listrik : listrik
hutang : hutang
bumn : bumn
progeamnya : progeamnya
pinjaman : pinjam
lunak : lunak
kamis : kamis
bunga : bunga
sejuta : juta
pulang : pulang
stgh : stgh
kumpul : kumpul
diluburin : diluburin
idup : idup
susah : susah
dibayarkan : bayar
boro : boro
puetinnting : puetinnting
entong : entong
ngelamar : ngelamar
suami : suami
lap : lap
sepatu : sepatu
gpp : gpp
jujur : jujur
berbahaya : bahaya
menular : tular
binatang : binatang
memanipulasi : manipulasi
seakan : akan
sedik : dik
jual : jual
termaksud : maksud
kejanggalan : janggal
udang : udang
dibalik : balik
bakwan : bakwan
buatin : buat

gampang : gampang
maksain : maksain
target : target
basi : basi
bosan : bosan
trend : trend
mercon : mercon
pleciden : pleciden
planciiiss : planciiiss
dibilang : bilang
kerjaannya : kerja
nakutin : nakutin
warga : warga
embeeerr : embeeerr
daaah : daaah
cmn : cmn
kepentingan : penting
alat : alat
kesehatan : sehat
banyaknya : banyak
pasien : pasien
atasi : atas
ribut : ribut
harapkan : harap
kel : kel
prioritas : prioritas
logika : logika
bermuatasi : bermuatasi
setiapa : setiapa
bermutasi : mutasi
vaksinnyapun : vaksin
berbeda : beda
efeknya : efek
mustahil : mustahil
logikanya : logika
mers : mers
kesimpulannya : simpul
perhitungkan : hitung
lakuin : lakuin
olahraga : olahraga
pagi : pagi
makanan : makan
mandi : mandi
sehari : hari
lupakan : lupa
menjalankan : jalan
ibadah : ibadah
shalat : shalat
adem : adem
ayem : ayem
jasmani : jasmani
rohani : rohani
cobanya : coba
kurun : kurun
bagus : bagus
hembusan : hembus
napas : napas
deteksi : deteksi
come : come
on : on
ngelawak : ngelaw

iming : iming
ksi : ksi
harapan : harap
krusuhan : krusuhan
dengarnya : dengar
lagii : lagi
melulu : melulu
dikeruk : keruk
luhut : luhut
pengalihan : alih
issue : issue
cristian : cristian
ronaldo : ronaldo
ori : ori
kemana : mana
modifikasi : modifikasi
dimajukan : maju
desember : desember
gara : gara
erick : erick
pulihkan : pulih
ekonomi : ekonomi
gelap : gelap
mata : mata
pemegang : pegang
transparan : transparan
kandungan : kandung
kebaikan : baik
senang : senang
terpamcing : terpamcing
soalmati : soalmati
takdir : takdir
nilihat : nilihat
bayak : bayak
masatakat : masatakat
mengeluh : keluh
isukorona : isukorona
produksi : produksi
kangen : kangen
sekolh : sekolh
banter : banter
triliun : triliun
pasienya : pasienya
kebetulankah : betul
menyebabkan : sebab
effect : effect
negative : negative
bertanggung : tanggung
process : process
keputusan : putus
dihisab : hisab
akhirat : akhirat
edit : edit
dibahas : bahas
viva : viva
le : le
dimmo : dimmo
bordirs : bordirs
fax : fax
income 

menguasai : kuasa
pakat : pakat
nyebut : nyebut
sara : sara
rasis : ras
sengaja : sengaja
sifatnya : sifat
targetnya : target
menembak : tembak
hutan : hutan
kalimantan : kalimantan
pelurumu : peluru
nyasar : nyasar
monas : monas
malaysia : malaysia
perumpamaan : umpama
peluru : peluru
dibersihkan : bersih
keliling : keliling
seasia : seasia
tenggara : tenggara
mengganti : ganti
rugi : rugi
infrastruktur : infrastruktur
rusak : rusak
luka : luka
kenepal : kenepal
disana : sana
kiasan : kias
disimpulkan : simpul
diobati : obat
jutaan : juta
ngeyel : ngeyel
otaknya : otak
disembunyikan : sembunyi
kolaps : kolaps
tergadai : gadai
iti : iti
diisi : isi
dinomer : dinomer
pabrik : pabrik
mesin : mesin
bertujuan : tuju
mengedit : edit
dna : dna
benda : benda
tali : tali
makluk : makluk
melewati : lewat
seleksi : seleksi
ketat : ketat
viks : viks
diberkati : kati
kebencian : benci
ubun : ubun
kuasai : kuasa
as : as
dilan : dil
rahasia : rahasia
sumbernya : sumber
asli : asli
negar : negar
adid

meng : meng
jawa : jawa
timur : timur
tolonglah : tolong
mrampas : mrampas
secepatnya : cepat
bissmillah : bissmillah
jadii : jadi
insha : insha
beraktivitas : aktivitas
pekalongan : kalong
mo : mo
wkkwkw : wkkwkw
gereja : gereja
doakan : doa
borong : borong
nikmati : nikmat
alhamdulilah : alhamdulilah
keluarkan : keluar
vaksinkan : vaksin
karyawan : karyawan
brsud : brsud
subang : subang
tugas : tugas
pengangkut : angkut
sampahnya : sampah
pemulasaraan : pemulasaraan
jenazah : jenazah
insentipnya : insentipnya
inssyha : inssyha
kabulkan : kabul
niat : niat
menolong : tolong
sungguh : sungguh
kepanjangan : panjang
pergi : pergi
incar : incar
taubat : taubat
insyhaf : insyhaf
kelakuan : laku
setan : setan
berdo : berdo
enyah : enyah
tercinta : cinta
oke : oke
tanggal : tanggal
kawatir : kawatir
lebat : lebat
mengguyur : guyur
aaaminnn : aaaminnn
aminin : aminin
walapun : walapun
percanya : perca
berjuang : juang
kemanusiaan : manusia
serapah : serapah
kalean : kalean
tenang : tenang
mem

mnjdi : mnjdi
alangkah : alangkah
baikny : baikny
dididik : didik
perbaikan : baik
menjadikan : jadi
pedoman : pedoman
baku : baku
berinteraksi : interaksi
sosial : sosial
dimohon : mohon
kebijkan : kebijkan
pelaksanaan : laksana
bodohkah : bodoh
kocak : kocak
nyiyiran : nyiyiran
liatnya : liat
canel : canel
plandemic : plandemic
berlanjut : lanjut
metal : metal
jadiin : jadiin
sono : sono
senengane : senengane
approve : approve
nanananana : nanananana
berkelanjutan : lanjut
dvaksin : dvaksin
tgu : tgu
sbulan : sbulan
reaksinya : reaksi
mntri : mntri
blkngan : blkngan
dimasukan : masuk
latihan : latih
membetuk : membetuk
antibodi : antibodi
lha : lha
dimasukin : dimasukin
tubuhnya : tubuh
sejenis : jenis
influenza : influenza
genetiknya : genetiknya
tipe : tipe
badanya : bada
letih : letih
lesu : lesu
kambuh : kambuh
nyantol : nyantol
permanen : permanen
kecapekan : cape
na : na
udzubillah : udzubillah
vaksinan : vaksin
lok : lok
njelasin : njelasin
just : just
transfer : transfer
fact

menyebarkan : sebar
menutup : tutup
nutupi : nutupi
kasusnya : kasus
diterima : terima
manusiawi : manusiawi
gugur : gugur
membuang : buang
mengahiri : mengahiri
atut : atut
merk : merk
ngeray : ngeray
bajakan : baja
tohhh : tohhh
cok : cok
pembekal : bekal
pastikan : pasti
memenuhi : penuh
ujian : uji
siip : siip
testing : testing
ground : ground
sarang : sarang
muasalny : muasalny
down : down
kk : kk
kokk : kokk
lhooo : lhooo
poopppl : poopppl
mm : mm
ukk : ukk
kkkk : kkkk
kompol : kompol
kmomkkokookokomkkkkokkmomo : kmomkkokookokomkkkkokkmomo
mookokmomokommomkomoommmomomkokomomkomlmkokokomommomoomoommoomm : mookokmomokommomkomoommmomomkokomomkomlmkokokomommomoomoommoomm
oo : oo
hja : hja
ular : ular
mulaijam : mulaijam
kkko : kkko
kkokok : kkokok
tank : tank
opo : opo
maneh : maneh
jl : jl
kkiiiiiiiiijjjjjjjj : kkiiiiiiiiijjjjjjjj
kmm : kmm
mjujjj : mjujjj
danpernah : danpernah
adayjj : adayjj
yh : yh
perbotol : botol
bati : bati
piro : piro
jajah : jajah
horee : horee
asik : asik
c

ngabalin : ngabalin
spy : spy
ketahuilah : tahu
menenangkan : tenang
ihtiar : ihtiar
berjalanndengan : berjalanndengan
diridhoi : diridhoi
mantu : mantu
cebongers : cebongers
tauladan : tauladan
rela : rela
berkorban : korban
mengerahkan : kerah
buzer : buzer
menggiring : giring
opini : opini
intimidasi : intimidasi
cakap : cakap
yra : yra
bedaaa : bedaaa
tertinggi : tinggi
mentrinya : tri
representasi : representasi
fadli : fadli
zon : zon
degil : degil
biofarma : biofarma
volunteer : volunteer
wiwi : wiwi
dateng : dateng
happy : happy
hilih : hilih
kinthil : kinthil
cemen : cemen
bat : bat
thug : thug
takuuuuuuu : takuuuuuuu
uttt : uttt
rezim : rezim
ajah : ajah
petingginya : petinggi
ptesidenya : ptesidenya
wakilnya : wakil
stafnya : staf
transeparan : transeparan
bisnes : bisnes
diwajibkan : wajib
undang : undang
videon : videon
penyampaianny : penyampaianny
rapi : rapi
mudh : mudh
kalem : kalem
dibarengi : bareng
relawanny : relawanny
deluan : delu
honest : honest
simulasi : simul

woii : woii
minggat : minggat
krng : krng
asupan : asupan
paten : paten
dipertemukan : temu
yotube : yotube
alogatrim : alogatrim
rakaat : rakaat
alfaitahah : alfaitahah
qulhu : qulhu
falaq : falaq
binnas : binnas
kusi : kus
tiupin : tiupin
tawwakal : tawwakal
blas : blas
musuh : musuh
pesex : pesex
sederhana : sederhana
diciptakan : cipta
merangsang : rangsang
menginfeksi : infeksi
potensi : potensi
merespon : merespon
patogen : patogen
dihadapinya : hadap
dikalahkan : kalah
pengibaratan : ibarat
penawar : tawar
ambigu : ambigu
assault : assault
rifle : rifle
gerakan : gera
separatis : separatis
vaxx : vaxx
doesn : doesn
antivaxxer : antivaxxer
bohonngg : bohonngg
okwlwlwl : okwlwlwl
gokil : gokil
orientasi : orientasi
keinget : keinget
saibou : saibou
wafat : wafat
istrimu : istri
sanak : sanak
perempuanmu : perempuan
jadian : jadi
gajelas : gajelas
yaudah : yaudah
unlike : unlike
anime : anime
immun : immun
kawai : kawai
campak : campak
polio : polio
mantullll : mantullll
edukasinya

In [35]:
# apply stemmed term to dataframe
def get_stemmed_term(document):
    return [term_dict[term] for term in document]

In [36]:
YOUTUBE_COMMENTS['comment_stemmed'] = YOUTUBE_COMMENTS['comment_stopword'].apply(get_stemmed_term)

## Join Token

In [37]:
YOUTUBE_COMMENTS['comment_joined'] = YOUTUBE_COMMENTS['comment_stemmed'].str.join(' ')

### REMOVE EMPTY DATAFRAMES

In [38]:
nan_value = float("NaN")
YOUTUBE_COMMENTS["comment_joined"].replace("", nan_value, inplace=True)
YOUTUBE_COMMENTS.dropna(subset = ["comment_joined"], inplace=True)

### EXPORT CSV

In [39]:
YOUTUBE_COMMENTS.to_csv("../dataset/ready/vaksin_comment_ready_13-12-2020_1.csv")
# CLEAN = pd.concat([YOUTUBE_COMMENTS['comment_stemmed'],pd.read_csv("vaksin_covid.csv",usecols=[3],nrows=576],axis=1)

In [40]:
print (YOUTUBE_COMMENTS)

         Video ID                                              Title  \
0     PwwL10ZlE6g          Jokowi: Vaksin Covid-19 Siap Januari 2021   
1     PwwL10ZlE6g          Jokowi: Vaksin Covid-19 Siap Januari 2021   
2     PwwL10ZlE6g          Jokowi: Vaksin Covid-19 Siap Januari 2021   
3     PwwL10ZlE6g          Jokowi: Vaksin Covid-19 Siap Januari 2021   
4     PwwL10ZlE6g          Jokowi: Vaksin Covid-19 Siap Januari 2021   
...           ...                                                ...   
1860  HjizKEjym0g  UPDATE KOMITE KPCPEN - Vaksin Covid-19, Protok...   
1861  HjizKEjym0g  UPDATE KOMITE KPCPEN - Vaksin Covid-19, Protok...   
1862  HjizKEjym0g  UPDATE KOMITE KPCPEN - Vaksin Covid-19, Protok...   
1863  HjizKEjym0g  UPDATE KOMITE KPCPEN - Vaksin Covid-19, Protok...   
1864  HjizKEjym0g  UPDATE KOMITE KPCPEN - Vaksin Covid-19, Protok...   

                                                Comment  Label  \
0                                    Vaksin tak berguna   -1.0   
1  